<a href="https://colab.research.google.com/github/amnbg/Digital-Signal-Processing/blob/main/DSP_Midterm_Requirement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ESP32 Code

In [ ]:
#include "WiFi.h"
#include <HTTPClient.h>
#include "time.h"
#include "DHT.h"

#define DHTPIN 23
#define DHTTYPE DHT11

const char* ntpServer1 = "pool.ntp.org";
const char* ntpServer2 = "time.nist.gov";
const long gmtOffset_sec = 28800;
const int daylightOffset_sec = 0;
float h = 0.0;
float t = 0.0;

char ssid[] = "HUAWEI-X57F";
char password[] = "kamotebbq";

DHT dht(23, DHT11);

String GOOGLE_SCRIPT_ID = "AKfycbwT9XBW5Q4AXvwUSoeKO9rUhsnPGwFz4dSwfun76MdMgR9FwJVhn_IezhAGCLKnzqY4yg";

void setup() {

  delay(1000);
  Serial.begin(115200);
  dht.begin();
  delay(1000);
  Serial.println();
  Serial.print("Connecting to wifi: ");
  Serial.println(ssid);
  WiFi.begin(ssid, password);
  while (WiFi.status() != WL_CONNECTED) {
    delay(500);
    Serial.print(".");

  }
  configTime(gmtOffset_sec, daylightOffset_sec, ntpServer1, ntpServer2);
}

void loop() {
  if (WiFi.status() == WL_CONNECTED) {
    SendDataToGoogleSheets();
  }
}

void SendDataToGoogleSheets() {
  // Read data from the DHT sensor
  float humi = dht.readHumidity();
  float temp = dht.readTemperature();

  if (isnan(humi) || isnan(temp)) {
    Serial.println(F("Failed to read from DHT sensor!"));
    return;
  }

  h = humi;
  t = temp;

  Serial.print(F("Humidity: "));
  Serial.print(h);
  Serial.print(F("%  Temperature: "));
  Serial.print(t);
  Serial.print(F("°C "));

  // Get the current time
  struct tm timeinfo;
  if (!getLocalTime(&timeinfo)) {
    Serial.println("Failed to obtain time");
    return;
  } else {
    char timeStringBuff[50];
    strftime(timeStringBuff, sizeof(timeStringBuff), "%m/%d/%Y_%H:%M:%S", &timeinfo);
    String asString(timeStringBuff);
    Serial.print("Time:");
    Serial.println(asString);

    // Create the URL for the Google Sheets script
    String urlFinal = "https://script.google.com/macros/s/" + GOOGLE_SCRIPT_ID + "/exec?" +
                     "date=" + asString + "&humi=" + String(h) + "&temp=" + String(t);
    Serial.print("POST data to spreadsheet:");
    Serial.println(urlFinal);

    // Send the data to Google Sheets
    HTTPClient http;
    http.begin(urlFinal.c_str());
    http.setFollowRedirects(HTTPC_STRICT_FOLLOW_REDIRECTS);
    int httpCode = http.GET();
    Serial.print("HTTP Status Code: ");
    Serial.println(httpCode);

    if (httpCode == 200) {
      delay(2000);
    }
    http.end();
    delay(595500);
  }
}

#Google AppScript Code

In [ ]:
var sheet_id = "1K0HeLmNkuk8qPa9J_hGO_KfmOzI1fnbaPE4wIE8y6JM";
var sheet_name = "Sheet1";
function doGet(e){
var ss = SpreadsheetApp.openById(sheet_id);
var sheet = ss.getSheetByName(sheet_name);
var humi = String(e.parameter.humi);
var temp = String(e.parameter.temp);
var date = String(e.parameter.date);
sheet.appendRow([date,humi,temp]);
}